# AWS - RDS MySQL
### Paquetes requeridos
Librería de Python necesaria para establecer conexión a una base de datos MySQL

In [ ]:
#!pip install PyMySQL

### Credenciales
Para acceder a la BD necesitaremos tres datos:
1. Endpoint/host: la dirección donde se encuentra el servidor
2. Usuario
3. Contraseña

In [ ]:
username = "Admin"
password = "Admin001"
host = "database-1.cwp3w8mjnv4z.eu-north-1.rds.amazonaws.com" 
port = 3306

### Conexion BD

In [ ]:
import pymysql

In [ ]:
'''
pymysql.cursors.DictCursor para que los resultados que devuelva sean diccionarios,
por defecto devuelve tuplas. Asi podemos acceder por clave a las columnas.
'''

db = pymysql.connect(host = host,
                     user = username,
                     password = password,
                     cursorclass = pymysql.cursors.DictCursor
)

# El objeto cursor es el que ejecutará las queries y devolverá los resultados

cursor = db.cursor()

### Version DB

In [ ]:
'''
Este es el engine version de la BD de AWS
fechtone trae la primera linea de la consulta
El execute() devuelve el numero de filas a las que ha afectado la query,
en este caso, devuelve una unica fila.

Execute se guarda en la conexion pero hasta que no hacemos commit
no se ejecutan las queries
de insert de datos y esas cosas...
'''

cursor.execute('SELECT VERSION()')
version = cursor.fetchone()
print(f'MySQL version: {version}')

### Creación de DB

In [ ]:
# Creamos una BD. Tenemos una instancia de MySQL, pero no una BD
# Los comandos de SQL  se suelen poner en mayuscula
create_db = '''CREATE DATABASE country_database'''
cursor.execute(create_db)

# Podemos eliminar la BD
# drop_db = '''DROP DATABASE country_database'''
# cursor.execute(drop_db)

# La volvemos a crear
# cursor.execute(create_db)
# el output es el numero de filas afectadas

In [ ]:
cursor.execute('SHOW DATABASES')
cursor.fetchall()

### Creación de tablas

In [ ]:
'''
Creamos una tabla
Esto da error porque no le hemos dicho al servidor qué BD
queremos usar

'''

# crear una tabla que se llame country

create_table = '''
CREATE TABLE country_database.country (
    id INT NOT NULL auto_increment,
    name TEXT,
    continent TEXT,
    population INT,
    gdp DOUBLE,
    primary key (id)
)
'''
cursor.execute(create_table)

### Seleccionar la BD

In [ ]:
# Para usar la BD  recien creada

cursor.connection.commit()
use_db = ''' USE country_database'''
cursor.execute(use_db)

In [ ]:
# checkear todas las tablas que tiene mi db
cursor.execute('SHOW TABLES')
cursor.fetchall()

In [ ]:

# drop_table = '''DROP TABLE country'''
# cursor.execute(drop_table)


### Insertar datos

In [ ]:
insert_data = '''
INSERT INTO country (name, continent, population, gdp)
VALUES ('Spain', 'Europe', 47, 1.74)
'''
cursor.execute(insert_data)

In [ ]:
name = 'France'
continent = 'Europe'
population = 54
gdp = 2

In [ ]:
insert_data = '''
INSERT INTO country (name, continent, population, gdp)
VALUES ('%s', '%s', '%s', '%s')
''' % (name,continent,population,gdp)

cursor.execute(insert_data)

### Leer datos

In [ ]:
sql = '''SELECT * FROM country'''
cursor.execute(sql)

In [ ]:
mi_lista = cursor.fetchall()
len(mi_lista)

In [ ]:
mi_lista

### Guardar los cambios
Hay que ejecutar el commit antes de cerrar la sesión de la BD para que se guarden todos los cambios

In [ ]:
db.commit()

### Excepciones
Se recomienda rodear este tipo de sentencias con un try/except, ya que suele ser software productivo y tiene que saber manejar fallos

In [ ]:
sql = '''SELECT * FROM countryy'''
cursor.execute(sql)
print(cursor.fetchall())
print('Proceso finalizado')

In [ ]:
try:
    sql = '''SELECT * FROM countryy'''
    cursor.execute(sql)
    print(cursor.fetchall())
except Exception as e:
    print(e)

print('Proceso finalizado')

### Insertar datos de un CSV

In [ ]:
# Creamos el CSV
import pandas as pd

df = pd.DataFrame({
    'name': ['Germany', 'Japan', 'Argentina', 'Brasil'],
    'continent' : ['Europe', 'Asia', 'America', 'America'],
    'population': [80, 126, 44, 209],
    'gdp': [4.3, 4.9, 0.5, 1.6]
})

df.head()

In [ ]:
df.to_csv('data/country_data.csv', sep=';')

In [ ]:
# leemos los datos
df = pd.read_csv('data/country_data.csv', sep=';', index_col=0)
df.head()

In [ ]:
#!pip install sqlalchemy #--upgrade 

In [ ]:
print(username)
print(password)
print(host)
print('country_database')

In [ ]:
from sqlalchemy import create_engine

# create sqlalchemy engine

engine = create_engine("mysql+pymysql://{user}:{pw}@{host}/{db}".format(user = username, pw = password, host = host, db = 'country_database'))
# engine = create_engine("mysql+pymysql://my_user:my_password@my_host/my_database")

In [ ]:
# insertamos todo el dataframe
df.to_sql(name='country', con=engine, if_exists= 'append', index=False)


In [ ]:
db.commit()

In [ ]:
# leemos los datos para comprobar que se han ingestado correctamente
sql = '''SELECT * FROM country'''
cursor.execute(sql)
mi_tabla = cursor.fetchall()
mi_tabla

In [ ]:
mi_tabla[4]['population']

In [ ]:
pd.DataFrame(mi_tabla)

### Cerrar la conexión cuando acabemos

In [ ]:
# Cerrar conexión mysql
db.close()

In [ ]:
# Cerrar conexión sqlalchuemy
engine.dispose()